In [ ]:
import numpy as np
import torch.optim as optim
import torch.nn as nn
import torch
from torch.utils.data import DataLoader
import plotly.graph_objects as go

device = torch.device('cpu')

# Driven damped oscillation
The equation of motion for the driven damped oscillation is given by:
$$F_0 cos(\omega{t}+\phi) = m\frac{d^2x}{dt^2}+\gamma\frac{dx}{dt}+kx$$

From this equation, we can define the physics-informed loss as:
$$(m\frac{d^2x}{dt^2}+\gamma\frac{dx}{dt}+kx-F_0 cos(\omega{t}+\phi))^2$$

For each training point, we calculate this physics-informed loss and take the average over all training sets.

Additionally, we introduce a loss term called the "trivial killer." In some cases (e.g., when the driven force is zero), the network may tend to output a straight line. While this conforms to physics at the time when the oscillator is at the origin and not oscillating, the residual loss will also remain zero.

It can sometimes be challenging for initial condition constraints to address this issue. Therefore, we introduce the trivial killer loss, which calculates the first and second derivatives of the network's output and adds them together as a loss term. This helps prevent the network's output from converging to local extrema.

After a certain point, when the network's output starts to oscillate, we must remove the trivial killer loss. This is because the actual solution will contain turning points (local extrema), and the trivial killer loss will hinder the network from learning the correct solution.


In [ ]:
# Implement sin activation function class 
# uses for sin function as a neural network activation function
# GOOGLE for SIREN network for details.

class SinActivation(nn.Module):
    def __init__(self):
        super(SinActivation, self).__init__()
        
    def forward(self, x):
        return torch.sin(10*x)

# Neural network class implement the SIREN network
class NeuralNet(nn.Module):

    def __init__(self, input_dimension, output_dimension, n_hidden_layers, neurons, regularization_param, regularization_exp, retrain_seed):
        super(NeuralNet, self).__init__()
        # Number of input dimensions n
        self.input_dimension = input_dimension
        # Number of output dimensions m
        self.output_dimension = output_dimension
        # Number of neurons per layer
        self.neurons = neurons
        # Number of hidden layers
        self.n_hidden_layers = n_hidden_layers
        # Activation function
        self.activation = SinActivation()
        self.regularization_param = regularization_param
        # Regularization exponent
        self.regularization_exp = regularization_exp
        # Random seed for weight initialization

        self.input_layer = nn.Linear(self.input_dimension, self.neurons)
        self.hidden_layers = nn.ModuleList([nn.Linear(self.neurons, self.neurons) for _ in range(n_hidden_layers - 1)])
        self.output_layer = nn.Linear(self.neurons, self.output_dimension)
        self.retrain_seed = retrain_seed
        # Random Seed for weight initialization
        self.init_xavier()

    def forward(self, x):
        # The forward function performs the set of affine and non-linear transformations defining the network
        # (see equation above)
        x = self.activation(self.input_layer(x))
        for k, l in enumerate(self.hidden_layers):
            x = self.activation(l(x))
        return self.output_layer(x)

    def init_xavier(self):
        torch.manual_seed(self.retrain_seed)

        def init_weights(m):
            if type(m) == nn.Linear and m.weight.requires_grad and m.bias.requires_grad:
                g = nn.init.calculate_gain('tanh')
                torch.nn.init.xavier_uniform_(m.weight, gain=g)
                # torch.nn.init.xavier_normal_(m.weight, gain=g)
                m.bias.data.fill_(0)

        self.apply(init_weights)

    def regularization(self):
        reg_loss = 0
        for name, param in self.named_parameters():
            if 'weight' in name:
                reg_loss = reg_loss + torch.norm(param, self.regularization_exp)
        return self.regularization_param * reg_loss


# Implement the PINN class for the driven oscillator problem
class DrivenOscillatorPinns:
    def __init__(self, n_int_):
        self.n_int = n_int_

        # Initial condition to solve driven oscillator
        self.initial_x = 3.0
        self.initial_v = 0.0
        self.init_cond = torch.tensor([self.initial_x,self.initial_v])
        
        # System parameters
        self.k = 50
        self.mass = 0.5
        self.c = 1
        
        # System parameters (Driven force)
        self.omega_d = 2
        self.phid = 0 
        self.F_o = 50
        
        # Loss weights
        self.initial_weight  = 1
        self.residual_weight = 5
        self.trivial_killer_weight = 1
        
        # Extrema of the solution domain (t) in [0,5]
        self.domain_extrema = torch.tensor([[0, 12]])  
        
        # Generator of Sobol sequences
        self.soboleng = torch.quasirandom.SobolEngine(dimension=1)
        # Initial training set
        self.training_set_int = self.assemble_datasets()

        # F Dense NN to approximate the solution of the underlying heat equation
        self.approximate_solution = NeuralNet(input_dimension=self.domain_extrema.shape[0], output_dimension=1,
                                            n_hidden_layers=2,
                                            neurons=100,
                                            regularization_param=0.,
                                            regularization_exp=0.,
                                            retrain_seed=5).to(device)
    
    # Function driving_term to compute the driving term at time t
    def driving_term(self, t):
        return self.F_o*torch.cos(self.omega_d*t + self.phid)
    # Function driven_oscillator to compute the analytical solution of the driven oscillator at time t
    def driven_oscillator(self,t):
        w0 = np.sqrt(self.k/self.mass)
        gamma = self.c/2/self.mass
        wprime = np.sqrt(w0**2 - gamma**2)
        
        print(f"wprime = {wprime}")
        A = self.F_o / self.mass / np.sqrt((w0**2 - self.omega_d**2)**2 + 4 * gamma**2 * self.omega_d**2 )
        
        print(self.k, self.mass, self.omega_d**2)
        phi = np.arctan(self.c * self.omega_d / (self.k - self.mass * self.omega_d**2)) - self.phid
        phih = np.arctan(wprime * (self.initial_x - A * np.cos(phi)) / (self.initial_v + gamma * (self.initial_x - A * np.cos(phi)) - A * self.omega_d * np.sin(phi) ) )
        
        Ah = (self.initial_x - A * np.cos(phi)) / np.sin(phih)
        
        x = Ah * np.exp(-gamma * t) * np.sin(wprime * t + phih) + A * np.cos(self.omega_d * t - phi)
        
        return x

    ################################################################################################
    # Function to linearly transform a tensor whose value are between 0 and 1
    # to a tensor whose values are between the domain extrema
    def convert(self, tens):
        assert (tens.shape[1] == self.domain_extrema.shape[0])
        return tens * (self.domain_extrema[0][1] - self.domain_extrema[0][0]) + self.domain_extrema[0][0]

    # Function to add interior points to the training set
    def add_interior_points(self):
        input_int = self.convert(self.soboleng.draw(self.n_int))
        output_int = torch.zeros((input_int.shape[0], 1))
        return input_int.to(device), output_int.to(device)

    # Function returning the training sets S_sb, S_tb, S_int as dataloader
    def assemble_datasets(self):
        input_int, output_int = self.add_interior_points() # S_int
        training_set_int = DataLoader(torch.utils.data.TensorDataset(input_int, output_int), batch_size=self.n_int, shuffle=False)

        return training_set_int
    

    ################################################################################################
    # Function to compute the terms required in the definition of the TEMPORAL boundary residual
    def initial_velocity(self,init_t):
        init_t.requires_grad = True
        pred_x_init = self.approximate_solution(init_t)
        pred_v_init = torch.autograd.grad(pred_x_init.sum(), init_t, create_graph=True)[0]
        
        return pred_v_init
    # Function returns the predicted initial position by the network
    # This function is used to compute the initial loss
    def initial_position(self,init_t):
        pred_x_init = self.approximate_solution(init_t)
        return pred_x_init

    # Function to compute the total loss (weighted sum of spatial boundary loss, temporal boundary loss and interior loss)
    def compute_loss(self, inp_train_int ,verbose=True):
        # Compute the predicted initial position and velocity
        init_t = torch.zeros(1).to(device)
        pred_init_position = self.initial_position(init_t)
        pred_init_velocity = self.initial_velocity(init_t)
        # Compute the predicted solution and its gradient w.r.t. t
        inp_train_int.requires_grad = True
        u = self.approximate_solution(inp_train_int)
        # Auto differentiation, use to compute the first and second derivatives of the network at each point of the training set
        grad_u_t = torch.autograd.grad(u.sum(), inp_train_int, create_graph=True)[0]
        grad_u_tt = torch.autograd.grad(grad_u_t.sum(), inp_train_int, create_graph=True)[0]
        # Compute the residual of the ODE
        residual = self.mass*grad_u_tt + self.c*grad_u_t + self.k*u  - self.driving_term(inp_train_int) 
        # Compute the loss from initial condition
        init_position_error = self.init_cond[0] - pred_init_position
        init_velocity_error = self.init_cond[1] - pred_init_velocity
        loss_initial_position = torch.mean(init_position_error ** 2)
        loss_initial_velocity = torch.mean(init_velocity_error ** 2)
        # Compute the loss from the ODE (Mean square error)
        loss_int = torch.mean(residual ** 2) 
        loss_tb = loss_initial_position + loss_initial_velocity
        # Trivial killer loss, This loss is used to kill trivial solutions
        # We did not use it in this example
        loss_trivial = (torch.reciprocal(torch.mean(u**2)) + torch.reciprocal(torch.mean(grad_u_t**2)) + torch.reciprocal(torch.mean(grad_u_tt**2)))
        # Compute the total loss (weighted sum of the three losses)
        loss = loss_tb * self.initial_weight  + loss_int * self.residual_weight + loss_trivial * self.trivial_killer_weight
        # Debug print
        if verbose: print("Total loss: ", round(loss.item(), 8), 
                        "| PDE Loss: ", round(loss_int.item(), 8), 
                        "| Initial Loss: ", round(loss_tb.item(), 8),
                        "| Trivial Loss: ", round(loss_trivial.item(), 8))
        return loss 

    ################################################################################################
    def fit(self, num_epochs, optimizer, verbose=True):
        history = list()
        for epoch in range(num_epochs):
            if verbose: print("################################ ", epoch, " ################################")
            
            for _, (inp_train_int, u_train_int) in enumerate(self.training_set_int):
                def closure():
                    optimizer.zero_grad()
                    loss = self.compute_loss(inp_train_int ,verbose=verbose)
                    loss.backward()
                    
                    history.append(loss.item())
                    return loss

                optimizer.step(closure=closure)

        print('Minimum Loss: ', min(history))

        return history


In [ ]:
# Number of interior points
n_int = 3000
pinn = DrivenOscillatorPinns(n_int)
# initialize the training set
input_int_, output_int_ = pinn.add_interior_points()
# Storage the training history (Loss)
hist = []

In [ ]:
# Plots the predicted solution before training
# For transcient state of the driven oscillator, we tooks more points
# since the change is more rapid in the transcient state
transcient_1 = torch.linspace(0, 1, 50).reshape(-1,1)
transcient_2 = torch.linspace(1, 2, 20).reshape(-1,1)
transcient   = torch.cat((transcient_1, transcient_2), 0)
# For steady state of the driven oscillator, we tooks less points
steady = torch.linspace(2, 12, 100).reshape(-1,1)
# combine transcient and steady time points
time_points = torch.cat((transcient, steady), 0)
output = pinn.approximate_solution(time_points)

fig = go.Figure()
fig.add_trace(go.Scatter(x=time_points.detach().numpy()[:,0], y=pinn.driven_oscillator(time_points).detach().numpy()[:,0], mode='markers', name='<b>Analytical Solution</b>', marker=dict(color='rgb(115, 187, 222)',size=7,symbol="star-diamond-open")))
fig.add_trace(go.Scatter(x=time_points.detach().numpy()[:,0], y=output.detach().numpy()[:,0], mode='markers', name='<b>PINN          Solution</b>', marker=dict(color='rgb(255, 169, 127)',size=6)))
fig.update_layout(xaxis_title=r'$\text{time}{(s)}$', yaxis_title=r'$\text{position}{(m)}$', template='plotly_dark',margin=dict(l=0, r=0, t=20, b=0),legend=dict(
    yanchor="top",font=dict(
            size=16,
            color="white"
        ),
    y=0.97,
    xanchor="right",
    x=0.97,
    bordercolor="white",
    borderwidth=1
))
fig.write_image('output/training_proc/Before_trainning.png',scale=5)


In [ ]:
total_epochs = 0

In [ ]:
# Setup learning rate and optimizer
l_rate = 0.001
optimizer_ADAM = optim.Adam(pinn.approximate_solution.parameters(),
                                lr=float(l_rate))


In [ ]:
# If it is necessary to lower the learning rate
# Run the following code
optimizer_ADAM.param_groups[0]['lr'] *= 0.1 

In [ ]:
epoch = 1000
for i in range(9):

    total_epochs += epoch

    # set loss weights to train the network
    pinn.initial_weight        = 600.0
    pinn.residual_weight       = 10000.0
    pinn.trivial_killer_weight = 0.0

    hist += pinn.fit(num_epochs=epoch,
                    optimizer=optimizer_ADAM,
                    verbose=True)
    # transcient_1 = torch.linspace(0, 1, 50).reshape(-1,1)
    # transcient_2 = torch.linspace(1, 2, 20).reshape(-1,1)
    # transcient   = torch.cat((transcient_1, transcient_2), 0)
    # steady = torch.linspace(2, 12, 100).reshape(-1,1)
    # # combine transcient and steady time points
    # time_points = torch.cat((transcient, steady), 0)
    # output = pinn.approximate_solution(time_points)

    # fig = go.Figure()
    # fig.add_trace(go.Scatter(x=time_points.detach().numpy()[:,0], y=pinn.driven_oscillator(time_points).detach().numpy()[:,0], mode='markers', name='<b>Analytical Solution</b>', marker=dict(color='rgb(115, 187, 222)',size=7,symbol="star-diamond-open")))
    # fig.add_trace(go.Scatter(x=time_points.detach().numpy()[:,0], y=output.detach().numpy()[:,0], mode='markers', name='<b>PINN          Solution</b>', marker=dict(color='rgb(255, 169, 127)',size=6)))
    # #set marker size
    # fig.update_xaxes(tickfont = dict(size=20),titlefont=dict(size=20))
    # fig.update_yaxes(tickfont = dict(size=20),titlefont=dict(size=20))
    # fig.update_layout(xaxis_title='time (s)', yaxis_title='position (m)', template='plotly_dark',margin=dict(l=0, r=0, t=20, b=0),legend=dict(
    #     yanchor="top",font=dict(
    #             size=16,
    #             color="white"
    #         ),
    #     y=0.97,
    #     xanchor="right",
    #     x=0.97,
    #     bordercolor="white",
    #     borderwidth=1
    # ))
    # fig.write_image(f'output/training_proc/epoch{total_epochs}_lr{l_rate}.png',scale=5)

In [ ]:
# Plots the loss function over epoches
fig = go.Figure()
fig.add_trace(go.Scatter(y=hist,mode='markers', name='Loss', marker=dict(color='rgb(115, 187, 222)',size=7)))
# set dark theme
# set x-axis to log scale
fig.update_layout(xaxis_title='epochs', yaxis_title='value', 
                  template='plotly_dark',margin=dict(l=0, r=0, t=20, b=0),
                  legend=dict(
                  yanchor="top",font=dict(
                        size=24,
                        color="white"
                    ),
               y=0.97,
               xanchor="right",
               x=0.97,
               bordercolor="white",
               borderwidth=1
            ),
                showlegend=True
                )
fig.update_xaxes(type="log",tickfont = dict(size=28),titlefont=dict(size=32),dtick = 1,tickformat='.1e')
fig.update_yaxes(type="log",tickfont = dict(size=28),titlefont=dict(size=32),dtick = 1,tickformat='.1e')
fig.write_image('output/Loss_function.png',scale=5,width= 1600, height= 300)

In [ ]:
# print out the predicted initial velocity and position
init_t = torch.zeros(1)
pred_init_position = pinn.initial_position(init_t)
pred_init_velocity = pinn.initial_velocity(init_t)
print("Initial position: ", pred_init_position.item())
print("Initial velocity: ", pred_init_velocity.item())

In [ ]:
# Randomly draw 10000 points to calculate the error between the analytical solution and the PINN solution
inputs = pinn.soboleng.draw(10000)
inputs = pinn.convert(inputs)
output = pinn.approximate_solution(inputs)
err_square = (pinn.driven_oscillator(inputs).detach().numpy()[:,0]-output.detach().numpy()[:,0])**2
err_mean = np.mean(err_square)
err_std = np.std(err_square)
print("Mean squared error: ", err_mean)
print("Standard deviation of squared error: ", err_std)
